In [0]:
-- =====================================================
-- Query 1: KPIs principales
-- =====================================================
-- Este query consolida en una sola consulta las métricas y dimensiones
-- necesarias para el cálculo de los KPIs principales del dashboard
-- (ventas, margen, transacciones, unidades y ticket promedio).
--
-- Se identificaron previamente los KPIs a mostrar y se decidió agruparlos
-- en un único query base para mejorar el desempeño, evitar múltiples
-- consultas repetidas y garantizar consistencia en los cálculos.
--
-- Nota:
-- Los filtros de fecha, región, formato y moneda se aplican de forma
-- dinámica desde el dashboard.
-- =====================================================

SELECT
    CAST(BUSINESS_DATE AS DATE) AS BUSINESS_DATE,
    date_format(CAST(BUSINESS_DATE AS DATE), 'dd-MM-yyyy') AS BUSINESS_DATE_FMT,
    STORE_REGION,
    STORE_FORMAT,
    CURRENCY,
    PRODUCT_CODE,
    PRODUCT_DESC,
    PRODUCT_DEPARTMENT,
    PRODUCT_CLASS,
    SUPPLIER_NAME,
    CHANNEL_NAME,
    STORE_ID,
    STORE_NAME,
    STORE_CITY,
    TOTAL_SALE_TAX,
    GROSS_MARGIN,
    PCT_GROSS_MARGIN,
    TOTAL_TRANSACTIONS,
    AVG_TICKET,
    TOTAL_UNITS
FROM workspace.operations.gold_sales_daily_summary

-- =====================================================
-- Query 2: Top 10 Productos por Ventas
-- =====================================================
-- Este query calcula el Top 10 de productos con mayor venta total
-- (TOTAL_SALE_TAX), agregando las ventas por producto y ordenando
-- de mayor a menor.
--
-- Se utilizan parámetros dinámicos para que el ranking se recalculé
-- automáticamente según los filtros seleccionados en el dashboard,
-- evitando resultados estáticos.
--
-- Parámetros utilizados:
-- - :region   → Filtra por región de la tienda
-- - :format   → Filtra por formato de tienda
-- - :currency → Filtra por moneda
-- - :fecha    → Rango de fechas seleccionado
--
-- Los filtros del dashboard permiten seleccionar múltiples valores,
-- por lo que los parámetros se reciben como arreglos (ARRAY).

-- La lógica ARRAY_SIZE / array_contains permite:
-- a. Aplicar el filtro cuando hay valores seleccionados
-- b. Retornar todos los registros cuando el filtro está vacío

-- Consideraciones:
-- - Solo se incluyen productos activos (STATUS = 'A')
-- =====================================================
SELECT 
    s.PRODUCT_DESC,
    SUM(s.TOTAL_SALE_TAX) AS TOTAL_SALE_TAX
FROM workspace.operations.gold_sales_daily_summary s
JOIN workspace.operations.gold_dim_product p
    ON s.PRODUCT_CODE = p.ITEM  
WHERE 
    (ARRAY_SIZE(:region) = 0 OR array_contains(:region, s.STORE_REGION))
    AND (ARRAY_SIZE(:format) = 0 OR array_contains(:format, s.STORE_FORMAT))
    AND (ARRAY_SIZE(:currency) = 0 OR array_contains(:currency, s.CURRENCY))
    AND (s.BUSINESS_DATE >= :fecha.min AND s.BUSINESS_DATE <= :fecha.max)
    AND p.STATUS = 'A'   
GROUP BY 1
ORDER BY TOTAL_SALE_TAX DESC
LIMIT 10;

-- =====================================================
-- Query 3: Top 10 Tiendas por Ventas
-- =====================================================
-- Este query calcula el Top 10 de tiendas con mayor venta total
-- (TOTAL_SALE_TAX)
--
-- Se utilizan parámetros dinámicos para que el ranking se recalculé
-- automáticamente según los filtros seleccionados en el dashboard,
-- evitando resultados estáticos.
--
-- Consideraciones:
-- - Solo se incluyen tiendas activas (STORE_ACTIVE = 'Y')
-- =====================================================
SELECT 
    s.STORE_NAME,
    s.STORE_CITY,
    SUM(s.TOTAL_SALE_TAX)/1000 AS TOTAL_SALE_TAX,
    SUM(s.GROSS_MARGIN)/1000 AS GROSS_MARGIN,     
    AVG(s.PCT_GROSS_MARGIN) AS PCT_MARGIN     
FROM workspace.operations.gold_sales_daily_summary s
JOIN workspace.operations.gold_dim_organization o
    ON s.STORE_ID = o.STORE_ID  
WHERE 
    (ARRAY_SIZE(:region) = 0 OR array_contains(:region, s.STORE_REGION))
    AND (ARRAY_SIZE(:format) = 0 OR array_contains(:format, s.STORE_FORMAT))
    AND (ARRAY_SIZE(:currency) = 0 OR array_contains(:currency, s.CURRENCY))
    AND (s.BUSINESS_DATE >= :fecha.min AND s.BUSINESS_DATE <= :fecha.max)
    AND o.STORE_ACTIVE = 'Y'  
GROUP BY 
    s.STORE_NAME, 
    s.STORE_CITY
ORDER BY TOTAL_SALE_TAX DESC
LIMIT 10;

-- =====================================================
-- Query 4: Top 5 Proveedores por Ventas
-- =====================================================
-- Este query calcula el Top 5 de proveedores con mayor venta total
-- (TOTAL_SALE_TAX)
--
-- Se utilizan parámetros dinámicos para que el ranking se recalculé
-- automáticamente según los filtros seleccionados en el dashboard,
-- evitando resultados estáticos.
--
-- Consideraciones:
-- - Solo se incluyen los proveedores activos (SUP_STATUS = 'A')
-- =====================================================
SELECT 
    s.SUPPLIER_NAME,
    SUM(s.TOTAL_SALE_TAX) AS TOTAL_SALE_TAX
FROM workspace.operations.gold_sales_daily_summary s
JOIN workspace.operations.gold_dim_supplier sup
    ON s.SUPPLIER_NAME = sup.SUP_NAME  
WHERE 
    (ARRAY_SIZE(:region) = 0 OR array_contains(:region, s.STORE_REGION))
    AND (ARRAY_SIZE(:format) = 0 OR array_contains(:format, s.STORE_FORMAT))
    AND (ARRAY_SIZE(:currency) = 0 OR array_contains(:currency, s.CURRENCY))
    AND (s.BUSINESS_DATE >= :fecha.min AND s.BUSINESS_DATE <= :fecha.max)
    AND sup.SUP_STATUS = 'A'  
GROUP BY 1
ORDER BY TOTAL_SALE_TAX DESC
LIMIT 5;

